<a href="https://colab.research.google.com/github/benza613/MS-Apriori-Algorithm/blob/master/DMTM_MSApriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab - not in local mode
%cd /content/drive/My Drive/DMTM

/content/drive/My Drive/DMTM


In [8]:
# list content of drive
%ls

assignments-1-2-3-testData/  IG_Sample1.txt  TestParameter.txt
CL_EVAL_sample1.txt          IG_Sample2.txt  TestTransaction.txt


In [139]:
# READ Transaction File 
transactions = []

fileData = open('assignments-1-2-3-testData/assign-1-testData/data-1/data-1.txt', "r+", encoding='utf-8-sig').readlines()
#fileData = open('assignments-1-2-3-testData/assign-1-testData/data-2/data-2.txt', "r+", encoding='utf-8-sig').readlines()
  
# remove all whitespaces, all newline chars, trailing commas that have no input after it
arrTransact = [line.replace(" ", "").rstrip("\n").rstrip(",") for line in fileData]

transactions = [list(map(int, line.split(","))) for line in arrTransact]

transactions

[[5, 8],
 [1, 2, 3, 6],
 [7, 9],
 [2, 4, 6],
 [1, 2, 3, 6, 7],
 [5],
 [8, 7],
 [9],
 [9],
 [8, 9, 7],
 [2, 5],
 [1, 4, 6, 7, 9],
 [7],
 [5, 9, 7],
 [5],
 [1, 8, 7, 9],
 [7, 9],
 [5, 9],
 [1, 2, 3, 6, 7],
 [7, 9]]

In [140]:
# READ Parameter file 
MIS= {}
SDC= 0 

mis_rest_val = 0.0

fileData = open('assignments-1-2-3-testData/assign-1-testData/data-1/para-1.txt', "r+", encoding='utf-8-sig').readlines()
#fileData = open('assignments-1-2-3-testData/assign-1-testData/data-2/para-2.txt', "r+", encoding='utf-8-sig').readlines()
arrParams = [line.replace(" ", "").rstrip("\n") for line in fileData]

# now we must read exactly-> MIS(n)=<float_value> and MIS(rest)=<float_value> and SDC=<float_value>
for param in arrParams:
    if param.find("MIS") > -1:
        # get indices
        key_start_idx = param.find("(")
        key_end_idx = param.find(")")
        val_start_idx = param.find("=")
        
        # extract & record dict keys and vals 
        key = param[key_start_idx + 1 : key_end_idx]
        val = float((param[val_start_idx + 1 :]))
        
        if key != 'rest':
            MIS[int(key)] = val
        else:
            # if REST -> we have to add this value for keys not in dict already later on 
            mis_rest_val = val
                    

    if param.find("SDC=") > -1:
        SDC = param[param.find("=") + 1 :]

        # assert if float works
        try:
            SDC = float(SDC)
        except ValueError:
            raise Exception("Error converting SDC value")

# now populate mis dict with pending item values using mis_rest_val
for t in transactions:
  for item in t:
    if item not in MIS.keys():
      MIS[int(item)] = mis_rest_val

MIS

{1: 0.28,
 2: 0.2,
 3: 0.12,
 4: 0.14,
 5: 0.27,
 6: 0.27,
 7: 0.27,
 8: 0.27,
 9: 0.27}

In [141]:
SDC

0.25

In [142]:
# Calculate M <- sort(I, MS) 
# where I == itemset in each transaction record
M = []

for t in transactions:
  t.sort()
  tmpD = []
  for item in t:
    tmpD.append((item,MIS[item]))

  tmpD.sort(key=lambda tup: tup[1])  # sorts in place
  M.append(tmpD)
  # sort tmpD based on tuple values
    
M

[[(5, 0.27), (8, 0.27)],
 [(3, 0.12), (2, 0.2), (6, 0.27), (1, 0.28)],
 [(7, 0.27), (9, 0.27)],
 [(4, 0.14), (2, 0.2), (6, 0.27)],
 [(3, 0.12), (2, 0.2), (6, 0.27), (7, 0.27), (1, 0.28)],
 [(5, 0.27)],
 [(7, 0.27), (8, 0.27)],
 [(9, 0.27)],
 [(9, 0.27)],
 [(7, 0.27), (8, 0.27), (9, 0.27)],
 [(2, 0.2), (5, 0.27)],
 [(4, 0.14), (6, 0.27), (7, 0.27), (9, 0.27), (1, 0.28)],
 [(7, 0.27)],
 [(5, 0.27), (7, 0.27), (9, 0.27)],
 [(5, 0.27)],
 [(7, 0.27), (8, 0.27), (9, 0.27), (1, 0.28)],
 [(7, 0.27), (9, 0.27)],
 [(5, 0.27), (9, 0.27)],
 [(3, 0.12), (2, 0.2), (6, 0.27), (7, 0.27), (1, 0.28)],
 [(7, 0.27), (9, 0.27)]]

In [143]:
import operator

# Calc L <- init-pass(M, T)
# basically counts of 1-itemsets in each transaction 
L = {}

# order indexer 
order = 0

for t in M:
  for tpl_item in t:
    item = tpl_item[0]
    item_mis = tpl_item[1]

    if item not in L.keys():
      L[item] = {
          'mis': item_mis,
          'count': 1,
          'order': order
      }

      order += 1
    else:
      L[item]['count'] += 1 

L

{1: {'count': 5, 'mis': 0.28, 'order': 5},
 2: {'count': 5, 'mis': 0.2, 'order': 3},
 3: {'count': 3, 'mis': 0.12, 'order': 2},
 4: {'count': 2, 'mis': 0.14, 'order': 8},
 5: {'count': 6, 'mis': 0.27, 'order': 0},
 6: {'count': 5, 'mis': 0.27, 'order': 4},
 7: {'count': 11, 'mis': 0.27, 'order': 6},
 8: {'count': 4, 'mis': 0.27, 'order': 1},
 9: {'count': 10, 'mis': 0.27, 'order': 7}}

In [144]:
# calculate F1 ==> Frequent itemsets of size 1
n = len(transactions)

F = {}
F['1'] = {}
F['1']['items'] = []

for i in L:
  if ( L[i]['count'] / n ) >= L[i]['mis'] :
    F['1']['items'].append(i)

F

{'1': {'items': [5, 3, 2, 7, 9]}}

In [145]:
import itertools 

def fn_Sup(val):
  return val/n 

def level2_candidate_gen(L, SDC):
  print('level2_candidate_gen')
  C2 = []

  for i in sorted(L, key = lambda name: L[name]['order']):

    if ( L[i]['count'] / n ) >= L[i]['mis'] :
      for h in sorted(L, key = lambda name: L[name]['order']):

        if L[h]['order'] != L[i]['order']:
          sdc_diff = fn_Sup(L[h]['count']) -  fn_Sup(L[i]['count'])
          
          if ( (L[h]['count']/n) >= L[i]['mis'] ) and ( abs(sdc_diff) <= SDC ) :

            # check if a different combination has not been inserted 
            match_found = False
            for c in C2:
              if set(c['items']) == set([i,h]):
                match_found = True

            if match_found == False:
              C2.append({'items': [i,h], 'counts': 0, 'tailcounts': 0})
            
  return C2


def MScandidate_gen(Fk, SDC, k):
  Ck = []

  for f1 in Fk:
    for f2 in Fk:
      #local var
      c =[]

      # doing this step to match pseudo code in textbook
      #f1['items'].sort()
      #f2['items'].sort()

      #print('f1 =>', f1['items'])
      #print('f2 =>', f2['items'])

      if set(f1['items'][:-1]) == set(f2['items'][:-1]):
        ik = f1['items'][-1]
        ik_dash = f2['items'][-1]
        sdc_diff = fn_Sup(L[ik]['count']) -  fn_Sup(L[ik_dash]['count'])

        #print('sdc',abs(sdc_diff) <= SDC)
        if (ik < ik_dash) and ( abs(sdc_diff) <= SDC ):
          c = f1['items']+f2['items'][-1:]
          Ck.append({'items':c, 'counts': 0, 'tailcounts': 0})
          
          
          for s in list(itertools.combinations(c, k)):
            if set([c[0]]).issubset(s) or ( L[c[0]]['mis'] ==  L[c[1]]['mis'] ) : 
              is_subset = False
              for f in Fk:
                if set(s).issubset(f['items']): 
                  is_subset = True
                  break;

              if is_subset == False:
                Ck.pop()
                break

  return Ck


In [146]:

k = 2 

C = {}

# loop K until the end of freq itemset iteration doesnt return empty set 
while len(F[str(k-1)]) > 0:
  if k == 2:
    C[2] = level2_candidate_gen(L, SDC)
  else:
    C[k] = MScandidate_gen(F[str(k-1)], SDC, k-1)
    #print(C[k])

  for t in transactions:
    for c in C[k]:
      if set(c['items']).issubset(t):
        c['counts'] += 1
      if set(c['items'][1:]).issubset(t):
        c['tailcounts'] += 1

  print(C[k])
  F[str(k)] = []
  for c in C[k]:
    if ( c['counts'] / n) >= L[c['items'][0]]['mis']:
      F[str(k)].append(c)
  
  #print('F')
  #print(F[str(k)])
      
  #print(k)
  k+= 1
 

 

level2_candidate_gen
[{'items': [5, 9], 'counts': 2, 'tailcounts': 10}, {'items': [3, 5], 'counts': 0, 'tailcounts': 6}, {'items': [3, 8], 'counts': 0, 'tailcounts': 4}, {'items': [3, 2], 'counts': 3, 'tailcounts': 5}, {'items': [3, 6], 'counts': 3, 'tailcounts': 5}, {'items': [3, 1], 'counts': 3, 'tailcounts': 5}, {'items': [2, 5], 'counts': 1, 'tailcounts': 6}, {'items': [2, 8], 'counts': 0, 'tailcounts': 4}, {'items': [2, 6], 'counts': 4, 'tailcounts': 5}, {'items': [2, 1], 'counts': 3, 'tailcounts': 5}, {'items': [2, 9], 'counts': 0, 'tailcounts': 10}, {'items': [7, 9], 'counts': 7, 'tailcounts': 10}]
[{'items': [3, 2, 6], 'counts': 3, 'tailcounts': 4}, {'items': [3, 1, 2], 'counts': 3, 'tailcounts': 3}, {'items': [3, 1, 6], 'counts': 3, 'tailcounts': 4}]
[{'items': [3, 1, 2, 6], 'counts': 3, 'tailcounts': 3}]
[]


In [151]:
# HERE we will print this output for u for F
STUDENT_ID = 'STUDENT_ID'

result = '(' + STUDENT_ID + '\n'
for i in F: 
  if int(i) == 1:
    if len( F[i]['items']) > 0:
      lgth = len(F[i]['items'])
      result += f'(Length-{i} { lgth } '
      result += '\n\t('
      result += ')\n\t('.join(map(str, F[i]['items']))
      result += ')\n'
      result += ') \n' # close length

  else:
    if len(F[i]) > 0:
      result += f'(Length-{i} {len(F[i])}'

      for pair in F[i]:
        result += '\n\t('+ ' '.join(map(str, pair['items']))
        result += ')'


    # for pair in F[i]:
    #   print(pair)
      
      result += '\n) \n)'
print(result)



(STUDENT_ID
(Length-1 5 
	(5)
	(3)
	(2)
	(7)
	(9)
) 
(Length-2 5
	(3 2)
	(3 6)
	(3 1)
	(2 6)
	(7 9)
) 
)(Length-3 3
	(3 2 6)
	(3 1 2)
	(3 1 6)
) 
)(Length-4 1
	(3 1 2 6)
) 
)


In [135]:
f = open('assignments-results/assign1/1_1_result.txt', "w")
f.write(result)
f.close()